In [5]:
pip install --upgrade pyarrow

  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/0e/2b/328c405ad7897db16f04c4998a3d514e5c311053a8dbf9cf575e4b3ff549/pyarrow-13.0.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/24.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.3 MB 1.1 MB/s eta 0:00:23
   ---------------------------------------- 0.3/24.3 MB 2.8 MB/s eta 0:00:09
    --------------------------------------- 0.5/24.3 MB 4.3 MB/s eta 0:00:06
   - -------------------------------------- 0.8/24.3 MB 4.0 MB/s eta 0:00:06
   - -------------------------------------- 1.0/24.3 MB 4.4 MB/s eta 0:00:06
   - -------------------------------------- 1.2/24.3 MB 4.4 MB/s eta 0:00:06
   -- ------------------------------------- 1.4/24.3 MB 4.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.6/24.3 MB 4.3 MB/s eta 0:00:06
   -- ------------------------------------- 1.7/24.3 MB 4.0 MB/s eta 0:00:06
   -- ----------------------

In [3]:
#Question 1: There's one feature with missing values. What is it?

import pandas as pd


file_path = 'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv('S:\ML Course\Homework Week 2\housing.csv')


# Filter the DataFrame to keep only the desired records
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]


columns_to_use = ['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
selected_columns_df = [columns_to_use]

# Use isna() or isnull() to check for missing values in each column and sum them up
missing_values = df.isna().sum()

# Print or display the columns with missing values
print("Columns with missing values:")
print(missing_values[missing_values > 0])

#Answer: total_bedrooms

Columns with missing values:
total_bedrooms    207
dtype: int64


In [4]:
#Question 2: What's the median (50% percentile) for variable 'population'?

import pandas as pd
import numpy as np

file_path = 'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv('S:\ML Course\Homework Week 2\housing.csv')



# Filter the DataFrame to keep only the desired records
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

columns_to_use = ['population']
selected_columns_df = [columns_to_use]

# Use isna() or isnull() to check for missing values in each column and sum them up
missing_values = df.isna().sum()

# Calculate the 50th percentile (median) using numpy
median = np.percentile(filtered_df['population'], 50)

# Print or display the median value
print('Median:', median)

## Answer: 1195

Median: 1195.0


In [5]:
#Question 3
#We need to deal with missing values for the column from Q1.
#We have two options: fill it with 0 or with the mean of this variable.
#Try both options. For each, train a linear regression model without regularization using the code from the lessons.
#For computing the mean, use the training only!
#Use the validation dataset to evaluate the models and compare the RMSE of each option.
#Round the RMSE scores to 2 decimal digits using round(score, 2)
#Which option gives better RMSE?

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# Load the dataset from the file path
file_path = r'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv(file_path)

# Filter the DataFrame to keep only the desired records
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Extract the 'population' column
selected_columns_df = filtered_df[['population']]

# Use isna() or isnull() to check for missing values in each column and sum them up
missing_values = filtered_df.isna().sum()

# Calculate the 50th percentile (median) using numpy
median = np.percentile(selected_columns_df['population'], 50)

# Shuffle the DataFrame using a random seed of 42
shuffled_df = filtered_df.sample(frac=1, random_state=42)

# Reset the index of the shuffled DataFrame
shuffled_df.reset_index(drop=True, inplace=True)

# Split the data into train (60%), validation (20%), and test (20%) sets
train_df, temp_df = train_test_split(shuffled_df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Apply the log transformation to the 'median_house_value' column
train_df['median_house_value'] = np.log1p(train_df['median_house_value'])

# One-hot encode the 'ocean_proximity' column for both options
train_df_option1_encoded = pd.get_dummies(train_df.copy(), columns=['ocean_proximity'], drop_first=True)
val_df_option1_encoded = pd.get_dummies(val_df.copy(), columns=['ocean_proximity'], drop_first=True)

train_df_option2_encoded = pd.get_dummies(train_df.copy(), columns=['ocean_proximity'], drop_first=True)
val_df_option2_encoded = pd.get_dummies(val_df.copy(), columns=['ocean_proximity'], drop_first=True)

# Define a SimpleImputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to fill missing values in the feature data
X_train_option1 = imputer.fit_transform(train_df_option1_encoded.drop(columns=['median_house_value']))
X_val_option1 = imputer.transform(val_df_option1_encoded.drop(columns=['median_house_value']))

X_train_option2 = imputer.fit_transform(train_df_option2_encoded.drop(columns=['median_house_value']))
X_val_option2 = imputer.transform(val_df_option2_encoded.drop(columns=['median_house_value']))

# Extract Target Variables
y_train_option1 = train_df_option1_encoded['median_house_value']
y_val_option1 = val_df_option1_encoded['median_house_value']

y_train_option2 = train_df_option2_encoded['median_house_value']
y_val_option2 = val_df_option2_encoded['median_house_value']

# Train Linear Regression Models
model_option1 = LinearRegression()
model_option1.fit(X_train_option1, y_train_option1)

model_option2 = LinearRegression()
model_option2.fit(X_train_option2, y_train_option2)

# Make Predictions on the Validation Set
y_pred_option1 = model_option1.predict(X_val_option1)
y_pred_option2 = model_option2.predict(X_val_option2)

# Calculate RMSE for Both Options
rmse_option1 = np.sqrt(mean_squared_error(y_val_option1, y_pred_option1))
rmse_option2 = np.sqrt(mean_squared_error(y_val_option2, y_pred_option2))

# Compare RMSE Scores and Determine the Better Option
print("Option 1 (Fill with 0) RMSE:", round(rmse_option1, 2))
print("Option 2 (Fill with Mean) RMSE:", round(rmse_option2, 2))

if rmse_option1 < rmse_option2:
    print("Option 1 (Fill with 0) performs better.")
else:
    print("Option 2 (Fill with Mean) performs better.")

# Answer: With Mean

Option 1 (Fill with 0) RMSE: 222188.81
Option 2 (Fill with Mean) RMSE: 222188.81
Option 2 (Fill with Mean) performs better.


In [6]:
#Question 4
#Now let's train a regularized linear regression.
#For this question, fill the NAs with 0.
#Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10].
#Use RMSE to evaluate the model on the validation dataset.
#Round the RMSE scores to 2 decimal digits.
#Which r gives the best RMSE?
#If there are multiple options, select the smallest r.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# Load the dataset from the file path
file_path = r'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv(file_path)

# Filter the DataFrame to keep only the desired records
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Shuffle the DataFrame using a random seed of 42
shuffled_df = filtered_df.sample(frac=1, random_state=42)

# Reset the index of the shuffled DataFrame
shuffled_df.reset_index(drop=True, inplace=True)

# Split the data into train (60%), validation (20%), and test (20%) sets
train_df, temp_df = train_test_split(shuffled_df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Log-transform the target variable 'median_house_value'
train_df['median_house_value'] = np.log1p(train_df['median_house_value'])
val_df['median_house_value'] = np.log1p(val_df['median_house_value'])

# One-hot encode the 'ocean_proximity' column
train_df_encoded = pd.get_dummies(train_df.copy(), columns=['ocean_proximity'], drop_first=True)
val_df_encoded = pd.get_dummies(val_df.copy(), columns=['ocean_proximity'], drop_first=True)

# Extract Features and Target Variables
X_train = train_df_encoded.drop(columns=['median_house_value'])
y_train = train_df_encoded['median_house_value']

X_val = val_df_encoded.drop(columns=['median_house_value'])
y_val = val_df_encoded['median_house_value']

# Use SimpleImputer to fill missing values with the mean (you can change the strategy if needed)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Define a list of regularization parameters (r) to try
regularization_params = [0, 1e-6, 1e-4, 1e-3, 0.01, 0.1, 1, 5, 10]

# Initialize variables to track the best RMSE and corresponding r
best_rmse = float('inf')
best_r = None

# Train Ridge Regression models for each value of r and find the best one
for r in regularization_params:
    model = Ridge(alpha=r)
    model.fit(X_train_imputed, y_train)

    # Make predictions on the validation set
    y_pred = model.predict(X_val_imputed)

    # Calculate RMSE and round to 2 decimal digits
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    # Print RMSE for the current value of r
    print(f"Ridge (alpha={r}): RMSE = {round(rmse, 2)}")

    # Update the best RMSE and corresponding r if necessary
    if rmse < best_rmse:
        best_rmse = rmse
        best_r = r

# Print the best value of r and corresponding RMSE
print(f"The best value of r is {best_r} with RMSE = {round(best_rmse, 2)}")

# Answer: 0.0001


Ridge (alpha=0): RMSE = 0.33
Ridge (alpha=1e-06): RMSE = 0.33
Ridge (alpha=0.0001): RMSE = 0.33
Ridge (alpha=0.001): RMSE = 0.33
Ridge (alpha=0.01): RMSE = 0.33
Ridge (alpha=0.1): RMSE = 0.33
Ridge (alpha=1): RMSE = 0.33
Ridge (alpha=5): RMSE = 0.33
Ridge (alpha=10): RMSE = 0.33
The best value of r is 0 with RMSE = 0.33


In [7]:
#Question 5
#We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
#Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
#For each seed, do the train/validation/test split with 60%/20%/20% distribution.
#Fill the missing values with 0 and train a model without regularization.
#For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
#What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
#Round the result to 3 decimal digits (round(std, 3))
#What's the value of std?


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Load the dataset
file_path = 'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv(file_path)

# EDA
# Look at the median_house_value variable
# Check if it has a long tail
median_house_value_tail = df['median_house_value'].tail()
print("Tail of median_house_value:\n", median_house_value_tail)

# Preparing the dataset
# Keep only records where ocean_proximity is either '<1H OCEAN' or 'INLAND'
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Use only the specified columns
selected_columns = [
    'latitude',
    'longitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income',
    'median_house_value'
]
df = filtered_df[selected_columns]

# Prepare and split the dataset
# Shuffle the dataset with seed 42
df_shuffled = df.sample(frac=1, random_state=42)

# Split the data into train/val/test sets (60%/20%/20%)
train, temp = train_test_split(df_shuffled, test_size=0.4, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Apply log transformation to 'median_house_value'
train['median_house_value'] = np.log1p(train['median_house_value'])
val['median_house_value'] = np.log1p(val['median_house_value'])
test['median_house_value'] = np.log1p(test['median_house_value'])


# Function to train and evaluate a linear regression model
def train_and_evaluate(df_train, df_val):
    X_train = df_train.drop(columns=['median_house_value'])
    y_train = df_train['median_house_value']

    X_val = df_val.drop(columns=['median_house_value'])
    y_val = df_val['median_house_value']

    # Create a pipeline with an imputer to handle missing values
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('linear_regression', LinearRegression())
    ])

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse


# List of seed values to try
seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

# Iterate through different seed values
for seed in seed_values:
    # Shuffle the dataset using the current seed
    df_shuffled = df.sample(frac=1, random_state=seed)

    # Split the data into train/val/test sets (60%/20%/20%)
    train, temp = train_test_split(df_shuffled, test_size=0.4, random_state=seed)
    val, test = train_test_split(temp, test_size=0.5, random_state=seed)

    # Apply log transformation to 'median_house_value'
    train['median_house_value'] = np.log1p(train['median_house_value'])
    val['median_house_value'] = np.log1p(val['median_house_value'])
    test['median_house_value'] = np.log1p(test['median_house_value'])

    # Train and evaluate a linear regression model
    rmse = train_and_evaluate(train, val)
    rmse_scores.append(rmse)

# Calculate the standard deviation of RMSE scores
std_deviation = np.std(rmse_scores)

# Round the standard deviation to 3 decimal digits
std_deviation_rounded = round(std_deviation, 3)

# Print the standard deviation
print(f"The standard deviation of RMSE scores is {std_deviation_rounded}")

# Answer: 0.005

Tail of median_house_value:
 20635    78100.0
20636    77100.0
20637    92300.0
20638    84700.0
20639    89400.0
Name: median_house_value, dtype: float64
The standard deviation of RMSE scores is 0.006


In [8]:
#Question 6
#Split the dataset like previously, use seed 9.
#Combine train and validation datasets.
#Fill the missing values with 0 and train a model with r=0.001.
#What's the RMSE on the test dataset?

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load the dataset
file_path = 'S:\ML Course\Homework Week 2\housing.csv'
df = pd.read_csv(file_path)

# EDA
# Look at the median_house_value variable
# Check if it has a long tail
median_house_value_tail = df['median_house_value'].tail()
print("Tail of median_house_value:\n", median_house_value_tail)

# Preparing the dataset
# Keep only records where ocean_proximity is either '<1H OCEAN' or 'INLAND'
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Use only the specified columns
selected_columns = [
    'latitude',
    'longitude',
    'housing_median_age',
    'total_rooms',
    'total_bedrooms',
    'population',
    'households',
    'median_income',
    'median_house_value'
]
df = filtered_df[selected_columns]

# Prepare and split the dataset
# Shuffle the dataset with seed 42
df_shuffled = df.sample(frac=1, random_state=42)

# Split the data into train/val/test sets (60%/20%/20%) using seed 9
train, temp = train_test_split(df_shuffled, test_size=0.4, random_state=9)
val, test = train_test_split(temp, test_size=0.5, random_state=9)

# Apply log transformation to 'median_house_value'
train['median_house_value'] = np.log1p(train['median_house_value'])
val['median_house_value'] = np.log1p(val['median_house_value'])
test['median_house_value'] = np.log1p(test['median_house_value'])

# Combine train and validation datasets
train_combined = pd.concat([train, val])

# Fill missing values with 0
train_combined.fillna(0, inplace=True)
test.fillna(0, inplace=True)

# Train a regularized linear regression model with r=0.001
X_train_combined = train_combined.drop(columns=['median_house_value'])
y_train_combined = train_combined['median_house_value']

X_test = test.drop(columns=['median_house_value'])
y_test = test['median_house_value']

# Train Ridge regression with r=0.001
ridge_model = Ridge(alpha=0.001)
ridge_model.fit(X_train_combined, y_train_combined)

# Make predictions on the test dataset
y_test_pred = ridge_model.predict(X_test)

# Calculate RMSE on the test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("RMSE on the test dataset:", rmse_test)

# Answer: 0.33

Tail of median_house_value:
 20635    78100.0
20636    77100.0
20637    92300.0
20638    84700.0
20639    89400.0
Name: median_house_value, dtype: float64
RMSE on the test dataset: 0.3432873502789519
